In [133]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
import numpy as np


In [156]:
df=pd.read_csv('data.csv')

In [158]:
df=df.drop_duplicates()
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])
# Create new columns
df['Transaction Hour'] = df['TransactionStartTime'].dt.hour
df['Transaction Day'] = df['TransactionStartTime'].dt.day
df['Transaction Month'] = df['TransactionStartTime'].dt.month
df['Transaction Year'] = df['TransactionStartTime'].dt.year

In [159]:
df['CountryCode'] = df['CountryCode'].astype(str)  # Ensure CountryCode is treated as a string

In [ ]:
categorical_df = df.select_dtypes(include=['object', 'category'])
categorical_df

Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId'],
      dtype='object')

In [161]:
numerical_df = df.select_dtypes(include=['number'])
numerical_df.columns

Index(['Amount', 'Value', 'PricingStrategy', 'FraudResult', 'Transaction Hour',
       'Transaction Day', 'Transaction Month', 'Transaction Year'],
      dtype='object')

In [162]:
# Define the columns for numerical and categorical features
numerical_features = ['Amount', 'Value', 'PricingStrategy', 'FraudResult', 'Transaction Hour','Transaction Day', 'Transaction Month', 'Transaction Year']
categorical_features = ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId','CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId','ProductCategory', 'ChannelId']

# Create the preprocessing pipeline for numerical features
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values with mean
    ('scaler', MinMaxScaler())                    # Normalize
])

# Create the preprocessing pipeline for categorical features
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))  # Impute missing values with mode
])

# Combine both pipelines into a single ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ]
)

# Fit and transform the data
df_transformed = preprocessor.fit_transform(df)

# Convert the transformed data back to a DataFrame
df_transformed = pd.DataFrame(df_transformed, columns=numerical_features + categorical_features)

In [163]:
df_transformed

,Amount,Value,PricingStrategy,FraudResult,Transaction Hour,Transaction Day,Transaction Month,Transaction Year,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId
0,0.092004,0.000101,0.5,0.0,0.086957,0.466667,0.909091,0.0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3
1,0.09191,0.000002,0.5,0.0,0.086957,0.466667,0.909091,0.0,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2
2,0.091958,0.00005,0.5,0.0,0.086957,0.466667,0.909091,0.0,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3
3,0.09375,0.002206,0.5,0.0,0.130435,0.466667,0.909091,0.0,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3
4,0.091853,0.000065,0.5,0.0,0.130435,0.466667,0.909091,0.0,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95657,0.09182,0.000101,0.5,0.0,0.391304,0.4,0.090909,1.0,TransactionId_89881,BatchId_96668,AccountId_4841,SubscriptionId_3829,CustomerId_3078,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2
95658,0.092004,0.000101,0.5,0.0,0.391304,0.4,0.090909,1.0,TransactionId_91597,BatchId_3503,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3
95659,0.09191,0.000002,0.5,0.0,0.391304,0.4,0.090909,1.0,TransactionId_82501,BatchId_118602,AccountId_4841,SubscriptionId_3829,CustomerId_3874,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2
95660,0.092188,0.000303,0.5,0.0,0.434783,0.4,0.090909,1.0,TransactionId_136354,BatchId_70924,AccountId_1346,SubscriptionId_652,CustomerId_1709,UGX,256,ProviderId_6,ProductId_19,tv,ChannelId_3


In [164]:
le = LabelEncoder()
# Fit and transform the specified columns
columns_to_encode = ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId','CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId','ProductCategory', 'ChannelId']
# Apply label encoding to each column individually
for column in columns_to_encode:
    df_transformed[column] = le.fit_transform(df_transformed[column])

In [165]:
# Normalization
min_max_scaler = MinMaxScaler()
df_normalized = pd.DataFrame(min_max_scaler.fit_transform(df_transformed), columns=df_transformed.columns)

In [167]:
df_normalized.to_csv('Final_preprocessed_data')